# 베이지안 최적화가 필요한 순간

- 가능한 최소의 시도로 최적의 답을 찾아야 할 경우

- 개별 시도가 너무 많은 시간/자원이 필요할 때

# HyperOpt 기본 실습

!pip install hyperopt

In [2]:
from hyperopt import hp

# -10 ~ 10까지 1간격을 가지는 입력 변수 x 집합값과 -15 ~ 15까지 1간격을 가지는 입력 변수 y 집합값 설정.
search_space = {'x' : hp.quniform('x', -10, 10, 1), 'y' : hp.quniform('y', -15, 15, 1) }
search_space

{'x': <hyperopt.pyll.base.Apply at 0x20bf0571b80>,
 'y': <hyperopt.pyll.base.Apply at 0x20bf300d760>}

In [4]:
from hyperopt import STATUS_OK

# 목적 함수를 생성. 입력 변수값과 입력 변수 검색 범위를 가지는 딕셔너리를 인자로 받고, 특정 값을 반환
def objective_func(search_space):
    x = search_space['x']
    y = search_space['y']
    retval = x**2 - 20*y
    
    return retval # return {'loss': retval, 'status':STATUS_OK}

In [5]:
from hyperopt import fmin, tpe, Trials
import numpy as np

# 입력 결괏값을 저장한 Trials 객체값 생성.
trial_val = Trials()

# 목적 함수의 최솟값을 반환하는 최적 입력 변숫값을 5번의 입력값 시도(max_evals=5)로 찾아냄.
best_01 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=5
               , trials=trial_val, rstate=np.random.default_rng(seed=0)
              )
print('best:', best_01)

100%|█████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 956.34trial/s, best loss: -224.0]
best: {'x': -4.0, 'y': 12.0}


In [37]:
trial_val = Trials()

# max_evals를 20회로 늘려서 재테스트
best_02 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=20
               , trials=trial_val) #rstate=np.random.default_rng(seed=0))
print('best:', best_02)

100%|███████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 932.24trial/s, best loss: -280.0]
best: {'x': 0.0, 'y': 14.0}


In [38]:
trial_val.results

[{'loss': 341.0, 'status': 'ok'},
 {'loss': -60.0, 'status': 'ok'},
 {'loss': -204.0, 'status': 'ok'},
 {'loss': 125.0, 'status': 'ok'},
 {'loss': 25.0, 'status': 'ok'},
 {'loss': 361.0, 'status': 'ok'},
 {'loss': -264.0, 'status': 'ok'},
 {'loss': 16.0, 'status': 'ok'},
 {'loss': -279.0, 'status': 'ok'},
 {'loss': -280.0, 'status': 'ok'},
 {'loss': 136.0, 'status': 'ok'},
 {'loss': 76.0, 'status': 'ok'},
 {'loss': 9.0, 'status': 'ok'},
 {'loss': -199.0, 'status': 'ok'},
 {'loss': 261.0, 'status': 'ok'},
 {'loss': -91.0, 'status': 'ok'},
 {'loss': -231.0, 'status': 'ok'},
 {'loss': 41.0, 'status': 'ok'},
 {'loss': 36.0, 'status': 'ok'},
 {'loss': -24.0, 'status': 'ok'}]

In [39]:
trial_val.vals

{'x': [-9.0,
  10.0,
  -6.0,
  5.0,
  -5.0,
  9.0,
  -6.0,
  -4.0,
  -1.0,
  0.0,
  -6.0,
  -4.0,
  -3.0,
  -9.0,
  9.0,
  7.0,
  3.0,
  9.0,
  6.0,
  -6.0],
 'y': [-13.0,
  8.0,
  12.0,
  -5.0,
  -0.0,
  -14.0,
  15.0,
  0.0,
  14.0,
  14.0,
  -5.0,
  -3.0,
  -0.0,
  14.0,
  -9.0,
  7.0,
  12.0,
  2.0,
  0.0,
  3.0]}

In [44]:
import pandas as pd

losses = [loss_dict['loss'] for loss_dict in trial_val.results]
losses

# DataFrame으로 생성
result_df = pd.DataFrame({'x': trial_val.vals['x'],
                           'y': trial_val.vals['y'],
                           'losses': losses
                          }
                        )
result_df

,x,y,losses
0,-9.0,-13.0,341.0
1,10.0,8.0,-60.0
2,-6.0,12.0,-204.0
3,5.0,-5.0,125.0
4,-5.0,-0.0,25.0
5,9.0,-14.0,361.0
6,-6.0,15.0,-264.0
7,-4.0,0.0,16.0
8,-1.0,14.0,-279.0
9,0.0,14.0,-280.0


# HyperOpt를 XGBoost 하이퍼 파라미터 튜닝에 적용

In [45]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [48]:
dataset = load_breast_cancer()

cancer_df = pd.DataFrame(data=dataset.data, columns=dataset.feature_names)
cancer_df['target']= dataset.target
X_features = cancer_df.iloc[:, :-1]
y_label = cancer_df.iloc[:, -1]

# 전체 데이터 중 80%는 학습용 데이터, 20%는 테스트용 데이터 추출
X_train, X_test, y_train, y_test=train_test_split(X_features, y_label,
                                         test_size=0.2, random_state=156 )

# 학습 데이터를 다시 학습과 검증 데이터로 분리 
X_tr, X_val, y_tr, y_val= train_test_split(X_train, y_train,
                                         test_size=0.1, random_state=156 )

In [49]:
from hyperopt import hp

# max_depth는 5에서 20까지 1간격으로, min_child_weight는 1에서 2까지 1간격으로
# colsample_bytree는 0.5에서 1사이, learning_rate는 0.01에서 0.2사이 정규 분포된 값으로 검색. 
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 20, 1),
                    'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),
                    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
                    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
               }

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

# 목적 함수 설정. 
# 추후 fmin()에서 입력된 search_space값으로 XGBClassifier 교차 검증 학습 후 -1* roc_auc 평균 값을 반환.  
def objective_func(search_space):
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth'])
                           , min_child_weight=int(search_space['min_child_weight'])
                            , colsample_bytree=search_space['colsample_bytree']
                            , learning_rate=search_space['learning_rate']
                           )
    
    # 3개 k-fold 방식으로 평가된 roc_auc 지표를 담는 list
    roc_auc_list= []
    
    # 3개 k-fold방식 적용 
    kf = KFold(n_splits=3)
    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train):
        # kf.split(X_train)으로 추출된 학습과 검증 index값으로 학습과 검증 데이터 세트 분리 
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]
        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행.
        xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric='auc'
                   , eval_set=[(X_tr, y_tr), (X_val, y_val)])
    
        # 1로 예측한 확률값 추출후 roc auc 계산하고 평균 roc auc 계산을 위해 list에 결과값 담음. 
        score = roc_auc_score(y_val, xgb_clf.predict_proba(X_val)[:, 1])
        roc_auc_list.append(score)
    
    # 3개 k-fold로 계산된 roc_auc값의 평균값을 반환하되, 
    # HyperOpt는 목적함수의 최소값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환. 
    return -1 * np.mean(roc_auc_list)